This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [2]:
import logging
from time import sleep

import numpy as np
from IPython.display import Image, clear_output, display
from pydrake.all import (
    ModelVisualizer,
    Parser,
    RobotDiagramBuilder,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.make_drake_compatible_model import MakeDrakeCompatibleModel
from manipulation.remotes import AddMujocoMenagerie
from manipulation.utils import ApplyDefaultVisualization

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

## MuJoCo Menagerie


In [ ]:
logging.getLogger("drake").setLevel(logging.ERROR)


def show_model(model: str):
    """For `model` as one of the keys in the `model_data` defined below, this will load the model in Drake and display it in Meshcat, and it will display the corresponding render from the Menagerie documentation."""
    model_url = f"package://mujoco_menagerie/{model}/{model_data[model][0]}"
    image_url = f"package://mujoco_menagerie/{model}/{model_data[model][1]}"

    builder = RobotDiagramBuilder()
    plant = builder.plant()
    parser = Parser(plant)
    package_map = parser.package_map()
    AddMujocoMenagerie(package_map)
    model_path = package_map.ResolveUrl(model_url)
    image_path = package_map.ResolveUrl(image_url)
    drake_model_path = model_path.replace(".xml", ".drake.xml")
    MakeDrakeCompatibleModel(model_path, drake_model_path)
    model_instances = parser.AddModels(drake_model_path)
    plant.Finalize()
    ApplyDefaultVisualization(builder.builder(), meshcat=meshcat)
    diagram = builder.Build()
    meshcat.Delete()
    simulator = Simulator(diagram)

    # Workaround for drake#22444: set the desired state to zero.
    for model_instance in model_instances:
        desired_state_port = plant.get_desired_state_input_port(model_instance)
        if desired_state_port.size() > 0:
            desired_state_port.FixValue(
                plant.GetMyContextFromRoot(simulator.get_context()),
                np.zeros(desired_state_port.size()),
            )

    diagram.ForcedPublish(simulator.get_context())
    display(Image(filename=image_path))


model_data = {
    "agility_cassie": ("scene.xml", "cassie.png"),  # Drake#22462
    "aloha": ("scene.xml", "aloha.png"),
    "anybotics_anymal_b": ("scene.xml", "anymal_b.png"),
    "anybotics_anymal_c": ("scene.xml", "anymal_c.png"),
    "berkeley_humanoid": (
        "scene.xml",
        "berkeley_humanoid.png",
    ),
    "bitcraze_crazyflie_2": ("scene.xml", "cf2.png"),
    "boston_dynamics_spot": ("scene_arm.xml", "spot.png"),
    #    "flybody": ("scene.xml", "flybody.png"), # Drake#22372
    "franka_emika_panda": ("scene.xml", "panda.png"),
    "franka_fr3": ("scene.xml", "fr3.png"),
    "google_barkour_v0": ("scene.xml", "barkour_v0.png"),
    "google_barkour_vb": ("scene.xml", "barkour_vb.png"),
    "google_robot": ("scene.xml", "robot.png"),
    "hello_robot_stretch": ("scene.xml", "stretch.png"),
    "hello_robot_stretch_3": ("scene.xml", "stretch.png"),
    "kinova_gen3": ("scene.xml", "gen3.png"),
    "kuka_iiwa_14": ("scene.xml", "iiwa_14.png"),
    "leap_hand": ("scene_right.xml", "right_hand.png"),
    "pal_talos": ("scene_motor.xml", "talos.png"),
    "pal_tiago": ("scene_motor.xml", "tiago.png"),
    "pal_tiago_dual": ("scene_motor.xml", "tiago_dual.png"),
    #    "realsense_d435i": ("d435i.xml", "d435i.png"), # Drake#22372
    "rethink_robotics_sawyer": ("scene.xml", "sawyer.png"),
    "robotiq_2f85": ("scene.xml", "2f85.png"),
    "robotiq_2f85_v4": ("scene.xml", "2f85.png"),
    "robotis_op3": ("scene.xml", "op3.png"),
    "shadow_dexee": ("scene.xml", "shadow_dexee.png"),
    "skydio_x2": ("scene.xml", "x2.png"),
    "trs_so_arm100": ("scene.xml", "so_arm100.png"),
    "trossen_vx300s": ("scene.xml", "vx300s.png"),
    "trossen_wx250s": ("scene.xml", "wx250s.png"),
    "ufactory_lite6": ("scene.xml", "lite6.png"),
    "ufactory_xarm7": ("scene.xml", "xarm7.png"),
    "unitree_a1": ("scene.xml", "a1.png"),
    "unitree_g1": ("scene_with_hands.xml", "g1.png"),
    "unitree_go1": ("scene.xml", "go1.png"),
    "unitree_go2": ("scene.xml", "go2.png"),
    "unitree_h1": ("scene.xml", "h1.png"),
    "unitree_z1": ("scene.xml", "z1.png"),
    "universal_robots_ur10e": ("scene.xml", "ur10e.png"),
    "universal_robots_ur5e": ("scene.xml", "ur5e.png"),
    "wonik_allegro": ("scene_left.xml", "allegro_hand.png"),
}

Show a particular model.

In [ ]:
if running_as_notebook:
    show_model("unitree_g1")

For fun, we can just loop through all of the models.

In [ ]:
if running_as_notebook:
    # Loop through and display all of the models, one at a time.
    for model in model_data.keys():
        clear_output(wait=True)
        print(f"Showing model: {model}")
        show_model(model)
        sleep(10)

## Model Visualizer

Note: some Menagerie models have implicit PD controllers; these will currently cause ModelVisualizer to fail, until we resolve https://github.com/RobotLocomotion/drake/issues/22444

In [ ]:
vis = ModelVisualizer(meshcat=meshcat)
parser = vis.parser()
AddMujocoMenagerie(parser.package_map())
model_path = parser.package_map().ResolveUrl(
    "package://mujoco_menagerie/agility_cassie/scene.xml"
)
drake_model_path = model_path.replace(".xml", ".drake.xml")
MakeDrakeCompatibleModel(model_path, drake_model_path)
vis.AddModels(drake_model_path)
vis.Run(loop_once=not running_as_notebook)